In [1]:
import pandas as pd
import os
import shutil
from Bio import SeqIO
from itertools import combinations
import subprocess

In [2]:
def remove_duplicates(strings):
    seen_pairs = set()    #########
    result = []
    for string in strings:
        parts = string.split('>')
        pair = tuple(sorted(parts))
        if pair not in seen_pairs:
            seen_pairs.add(pair)
            result.append(string)
    return result

def remove_duplicates2(strings):
    result = []
    for string in strings:
        parts = string.split('>')
        if len(parts) == 2 and parts[0] != parts[1]:
            result.append(string)
    return result


计算HGT Frequent Percentage

In [3]:
blastn_path='/Users/jiachenghao/projects/Gal/eLife初稿/eLife_修回/Question4/Step3_Cal_HGT_CrossRegion/blastn'
work_path='/Users/jiachenghao/projects/Gal/eLife初稿/eLife_修回/Question4/Step3_Cal_HGT_CrossRegion'

In [4]:
working_directory=blastn_path+'/OUT'
command = ["cat", "*.txt", ">", "Final_blastn_merged.txt"]
result = subprocess.run(" ".join(command), shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=working_directory)

if result.returncode == 0:
    print("File Combined")
    print(result.stdout.decode())
else:
    print("File Combined Failed")
    print(result.stderr.decode())

shutil.move(blastn_path+'/OUT/Final_blastn_merged.txt',work_path)

File Combined



'/Users/jiachenghao/projects/Gal/eLife初稿/eLife_修回/Question4/Step3_Cal_HGT_CrossRegion/Final_blastn_merged.txt'

In [5]:
file=pd.read_csv('Final_blastn_merged.txt',sep='\t',header=None)
column=['name1','name2','name3','name4','name5','name6','name7','name8','name9','name10',
        'name11','name12','name13','name14']
file.columns=column
file_length2=file.shape[0]
x_col=['>']*file_length2
file['x']=x_col
file['y']=file.apply(lambda row: ''.join(row[['name1', 'x', 'name2']]), axis=1)

file_name1=list(set(file['name1'].to_list()))

In [6]:
file

,name1,name2,name3,name4,name5,name6,name7,name8,name9,name10,name11,name12,name13,name14,x,y
0,SAL_PA7976AA_0,SAL_PA7725AA_3,100.000,837,0,0,1,837,1,837,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,0.0,1546,>,SAL_PA7976AA_0>SAL_PA7725AA_3
1,SAL_PA7976AA_0,SAL_PA7695AA_0,100.000,837,0,0,1,837,1,837,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,0.0,1546,>,SAL_PA7976AA_0>SAL_PA7695AA_0
2,SAL_PA7976AA_0,SAL04027_4,99.881,837,1,0,1,837,1,837,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,0.0,1541,>,SAL_PA7976AA_0>SAL04027_4
3,SAL_PA7976AA_0,SAL03319_1,99.881,837,1,0,1,837,1,837,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,0.0,1541,>,SAL_PA7976AA_0>SAL03319_1
4,SAL_PA7976AA_0,SAL03320_2,99.881,837,1,0,1,837,1,837,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,0.0,1541,>,SAL_PA7976AA_0>SAL03320_2


In [7]:
file.shape

(5, 16)

In [8]:
All_Y=[]
All_verify=[]
Check_MAX=[]

for i in file_name1:
    temp_file=file[file['name1']==i]

    max_value = 837   #### Coverage == 100%  ### It is important here to manually enter the length of the ARGs！！！
    Check_MAX.append(max_value)

    temp_file3=temp_file[temp_file['name4']==max_value]
    temp_file3=temp_file3[temp_file3['name3'] ==  100] 
    
    ready_to_depulicate=temp_file3['y'].to_list()
    verfy=temp_file3['name4'].to_list()
    for ii in ready_to_depulicate:
        All_Y.append(ii)
    for iii in verfy:
        All_verify.append(iii)


print('The 100% Coverage is', list(set(Check_MAX))[0])
if len(list(set(Check_MAX))) > 1:
    print('Warning: the 100% Identity is not only.')

The 100% Coverage is 837


In [9]:
filtered_1=remove_duplicates(All_Y)
filtered_2=remove_duplicates2(filtered_1)
temp_out=pd.DataFrame(filtered_2)
temp_out.to_csv('HGT_Detailed.csv',index=False)

In [10]:
All_Y_Motified=[]
for i in All_Y:
    i=i.split('>')
    i_motifi_1=i[0]
    last_underscore_index=i_motifi_1.rfind('_')
    i_motifi_1=i_motifi_1[:last_underscore_index]

    i_motifi_2=i[-1]
    last_underscore_index=i_motifi_2.rfind('_')
    i_motifi_2=i_motifi_2[:last_underscore_index]

    Motified_i=i_motifi_1+'>'+i_motifi_2
    All_Y_Motified.append(Motified_i)

In [11]:
filtered_1=remove_duplicates(All_Y_Motified)
filtered_2=remove_duplicates2(filtered_1)
temp_out=pd.DataFrame(filtered_2)
temp_out.to_csv('HGT_Strain_Non_Detailed.csv',index=False)

In [12]:
# final_all_all=file_length*file_length    #######Bug2
final_all_count=len(filtered_2)
print(final_all_count)
final_all_all=file.shape[0] ###########
print(final_all_all) 
HGT_Frequency=final_all_count/final_all_all
HGT_Frequency=round(HGT_Frequency,2)
print('HGT CrossRegion is',HGT_Frequency)

2
5
HGT CrossRegion is 0.4
